In [1]:
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data
import datetime

### Defined the directory and model parameter

In [2]:
log_directory = "strategy_log/"
file_directory = "data/"

In [3]:
look_back = 3
portfolio_size = 10

### Extract the data from Yahoo Finance

In [4]:
with open(file_directory+"yahoo_finance_crypto_list.txt") as f:
    
    ticker_list = f.readlines()
    
    ticker_list = [ticker.strip() for ticker in ticker_list]

In [5]:
start_date = "2021-01-05"
end_date = "2021-01-18"

In [ ]:
start_date = (datetime.datetime.today()-datetime.timedelta(days=look_back)).strftime("%Y-%m-%d")
end_date = (datetime.datetime.today()).strftime("%Y-%m-%d")

In [6]:
price_dic = {}
error_list = []

for ticker in ticker_list:
    
    try:
        price_df = get_data(ticker=ticker,
                        start_date=start_date,
                        end_date=end_date,
                        index_as_date=True,
                        interval="1d")
        
        price_dic[ticker] = price_df
    
    except:
        
        print("Error.")
        error_list.append(ticker)
    
    #time.sleep(1)

Error.
Error.
Error.
Error.
Error.


In [7]:
price_df_list = [price_dic[ticker] for ticker in ticker_list if ticker not in error_list]
price_df = pd.concat(price_df_list,axis=0)

In [8]:
price_df

,open,high,low,close,adjclose,volume,ticker
2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,33992.429688,6.754732e+10,BTC-USD
2021-01-06,34013.613281,36879.699219,33514.035156,36824.363281,36824.363281,7.528943e+10,BTC-USD
2021-01-07,36833.875000,40180.367188,36491.191406,39371.042969,39371.042969,8.476214e+10,BTC-USD
2021-01-08,39381.765625,41946.738281,36838.636719,40797.609375,40797.609375,8.810752e+10,BTC-USD
2021-01-09,40788.640625,41436.351562,38980.875000,40254.546875,40254.546875,6.198416e+10,BTC-USD
...,...,...,...,...,...,...,...
2021-01-14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,RSTR-USD
2021-01-15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,RSTR-USD
2021-01-16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,RSTR-USD
2021-01-17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,RSTR-USD


### Filter the universe by average transaction volume

In [11]:
average_volume = price_df[["volume","ticker"]].groupby("ticker").mean()
volume_threshold = average_volume.describe().loc["25%",:][0]
universe_list = list(average_volume[average_volume>volume_threshold].index)

In [12]:
price_df_list = [price_dic[ticker] for ticker in universe_list]
price_df = pd.concat(price_df_list,axis=0)

### Calculate Signal

In [13]:
price_df["date"] = price_df.index
price_wide = price_df.pivot_table(values="close",columns="ticker",index="date")


signal = price_wide.pct_change(look_back)
signal = signal.iloc[-1,:].dropna()

In [14]:
signal

ticker
ABBC-USD   -0.085530
ACH-USD     0.373561
ACT-USD     0.019229
ADA-USD     0.227738
ADK-USD    -0.005630
              ...   
ZEN-USD     0.290404
ZIL-USD     0.182737
ZNN-USD    -0.116627
ZRX-USD     0.134655
ZYN-USD    -0.196423
Name: 2021-01-18 00:00:00, Length: 365, dtype: float64

### Create Portfolios

In [15]:
top_portfolio = signal.nlargest(portfolio_size)
bottom_portfolio = signal.nsmallest(portfolio_size)

top_portfolio = pd.DataFrame(top_portfolio).T
bottom_portfolio = pd.DataFrame(bottom_portfolio).T

In [16]:
bottom_portfolio

ticker,AIB-USD,COMP1-USD,COMP2-USD,COLX-USD,MINT-USD,DUN-USD,VIN-USD,BST-USD,ECA-USD,ARRR-USD
2021-01-18,-0.785313,-0.678571,-0.678571,-0.665739,-0.502512,-0.484271,-0.436422,-0.418341,-0.347175,-0.333798


In [17]:
top_portfolio

ticker,ALIAS-USD,MTC1-USD,MTC2-USD,BONO-USD,PHA-USD,VERI-USD,RBY-USD,GCC1-USD,GCC2-USD,MBC-USD
2021-01-18,1.996485,1.579798,1.579798,1.261967,1.219795,0.967632,0.890407,0.799602,0.799602,0.788112


### Write the Log

In [19]:
top_portfolio.to_csv(log_directory+"top_portfolio"+end_date+".csv")
bottom_portfolio.to_csv(log_directory+"bottom_portfolio"+end_date+".csv")

In [18]:
end_date

'2021-01-18'